In [1]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline
# import functions from the modsim.py module
from modsim import *

In [29]:
# Constants
### Note: time is in seconds

# math constants
c = 3*10**(8)
sigma = 5.6703*10**(-8)
k = 1.380649*10**(-23) 
avogadro = 6.022140857*10**(23)
pi = 3.1415926535

# sun stats
solar_mass = 1.98847*10**40
solar_density = 1410

# state and system
t_0 = 0
t_end = 2.4e15
T_init = 25000
T_final = 273.15
step = 1e14

# conversions
s_in_year = 31557600
carbon_mass_per_mol = 12.011 *10**(-3)
star_mass = 1.018 * solar_mass
density = 92000*solar_density

In [30]:
# Physics Formulas

def surface_area(mass, density):
    """
    this function calculates the surface area for a sphere of a given material and uniform composition
    mass is the mass of the sphere
    density is the duniform density of the material (kg/m^3)
    constants is a system object containing fundamental constants
    """
    
    #calculates the volume of the material
    volume = mass/density
    
    #converts the volume to a sphere and finds the radius
    radius = (volume/(4/3)/pi)**(1/3)
    
    #returns the surface area of the sphere
    return 4*pi*(radius**2)

def num_particles(m, M):
    """
    this function finds the number of particles from molar molar mass and mass
    m is mass of the object
    M is the molar mass of the object
    """
    return avogadro * m / M

def star_energy_init():
    return T_init*(((star_mass/carbon_mass_per_mol)*avogadro)*k)

In [31]:
# State
"""
this code creates a the state of the system
T_star is the themperature of the star
star_energy is the internal energy of the star
star_mass is the mass of the star
surf_area is the surface area of the star
"""

#creates the state of the function
state = State(T_star = T_init,
            star_energy = star_energy_init(),
            star_mass = star_mass,
            surf_area = surface_area(star_mass, density),
            N = num_particles(star_mass, carbon_mass_per_mol))

In [32]:
#creates the system of the white dwarf

"""
creates a system with innital values and constants
init is the innital state function
emmisivity is an innate property of the materials involved. 
White dwarfs emmit radiation in a near perfict manner which correlates to them having an emmisivity value of very close to 1
density is the density of the white dwarf in kg/m^3 and is found by multiplying the density of the sun times 92,000
"""

system = System(init = state,
             t_0 = t_0,
             t_end = t_end,
             T_env  = 2.7,
             T_init = T_init,
             sigma = sigma,
             emissivity  = 1,
             k = k,
             density = density)

In [33]:
#Update Function
def update_function(state, t, system):
    unpack(system)
    t_s, s_e, s_m, s_a, n = state
    
    part = 3 *  emissivity * sigma * init.surf_area / (init.N * k)
    
    dT = - (1/3) * part * ((part * t + (1/T_final)**3) ** (-4/3)) * s_in_year
    t_s += dT * step 
    
    return State(T_star = t_s, 
                 star_energy = s_e,
                 star_mass = s_m,
                 surf_area= s_a,
                 N= n)

In [34]:
def run_simulation(system, update_func):
    """Runs a simulation of the system.
        
    system: System object
    update_func: function that updates state
    
    returns: TimeFrame
    """
    unpack(system)
    
    frame = TimeFrame(columns=init.index)
    frame.row[t_0] = init
    t = t_0
    for t in linspace(int(t_0), int(t_end), num = t_end/step + 1):
        frame.row[int(t+step)] = update_func(frame.row[int(t)], t, system)
        t += step
    return frame

In [35]:
 frame = TimeFrame(columns=state.index)
 frame.row[t_0] = state
 frame.row[t_0+step] = update_function(frame.row[t_0], t_0, system)
 frame

,T_star,star_energy,star_mass,surf_area,N
0,25000.00000,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
100000000000000,24003.56183,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66


In [36]:
run_simulation(system, update_function)

,T_star,star_energy,star_mass,surf_area,N
0,25000.000000,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
100000000000000,24003.561830,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
200000000000000,23007.124120,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
300000000000000,22010.686871,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
400000000000000,21014.250083,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
500000000000000,20017.813755,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
600000000000000,19021.377888,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
700000000000000,18024.942482,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
800000000000000,17028.507537,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
900000000000000,16032.073053,3.503175e+47,2.024262e+40,1.401707e+22,1.014936e+66
